In [1]:
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

In [53]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.io import curdoc
from bokeh.plotting import figure, show, save#, ColumnDataSource
from bokeh.models import ColumnDataSource, Slider, TextInput, HoverTool
from bokeh.layouts import widgetbox, row

In [3]:
trainlines = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='railway')

In [4]:
clyde = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='river')

In [5]:
#stations  contains schematics geometry and calculated price
stations = gpd.read_file(r'd:\viz\trainlines\derived\New folder\station_price.gpkg', layer='price_dz_points')

In [6]:
stations['name'] = stations['name'].str.upper()

In [7]:
stations['mean'] = pd.to_numeric(stations['mean']).round()

In [8]:
subway  = gpd.read_file(r'D:\viz\trainlines\viz_layers.gpkg', layer='subway')

In [9]:
subway['name'] = subway['name'].str.upper()

In [10]:
# update CRS for all layers (as shp were in BNG )
trainlines.crs = {'init' :'epsg:27700'} 
print('Trainlines: ', trainlines.crs)

#stations.crs = {'init' :'epsg:27700'} 
print('Stations : ', stations.crs)

#subway.crs = {'init' :'epsg:27700'}  
print('Subway : ',subway.crs)

#clyde.crs= {'init' :'epsg:27700'} 
print('Clyde : ', clyde.crs)

Trainlines:  {'init': 'epsg:27700'}
Stations :  {'init': 'epsg:27700'}
Subway :  {'init': 'epsg:27700'}
Clyde :  {'init': 'epsg:27700'}


In [11]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [12]:
def getLineCoords(row, geom, coord_type):
    """Returns a list of coordinates ('x' or 'y') of a LineString geometry"""
    if coord_type == 'x':
        return list( row[geom].coords.xy[0] )
    elif coord_type == 'y':
        return list( row[geom].coords.xy[1] )

In [13]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

**Let’s now apply the functions that we have created and parse the x and y coordinates for all of our datasets.**

In [14]:
# Get the Polygon x and y coordinates
#grid['x'] = grid.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
#grid['y'] = grid.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

# Calculate x and y coordinates of the line
trainlines['x'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
trainlines['y'] = trainlines.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)


clyde['x'] = clyde.apply(getLineCoords, geom='geometry', coord_type='x', axis=1)
clyde['y'] = clyde.apply(getLineCoords, geom='geometry', coord_type='y', axis=1)

In [15]:
# Calculate x and y coordinates of the points
subway['x'] = subway.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
subway['y'] = subway.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

stations['x'] = stations.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
stations['y'] = stations.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)

**Let’s now convert our GeoDataFrames into Bokeh ColumnDataSources (without geometry columns)**

In [16]:
# Make a copy, drop the geometry column and create ColumnDataSource
tr_df = trainlines.drop('geometry', axis=1).copy()
trsource = ColumnDataSource(tr_df)

# Make a copy, drop the geometry column and create ColumnDataSource
cl_df = clyde.drop('geometry', axis=1).copy()
clsource = ColumnDataSource(cl_df)


# Make a copy, drop the geometry column and create ColumnDataSource
s_df = subway.drop('geometry', axis=1).copy()
ssource = ColumnDataSource(s_df)

# Make a copy, drop the geometry column and create ColumnDataSource
st_df = stations.drop('geometry', axis=1).copy()
stsource = ColumnDataSource(st_df)

## Data visualising

In [17]:
# Initialize our figure
p = figure(title="Stations Glasgow",sizing_mode='scale_width')

In [165]:
# Add clyde on top of the same figure
p.multi_line('x', 'y', source=clsource, color="#5e96cb", line_width=14, line_cap ='round', line_join = 'round')
# Add clyde additional styling 
##TO-DO find a better way to achieve similar styling result
p.multi_line('x', 'y', source=clsource, color="white", line_width=12, line_cap ='round', line_join = 'round')
p.multi_line('x', 'y', source=clsource, color="#98bcde", line_width=10, line_cap ='round', line_join = 'round')

GlyphRenderer(id='4664', ...)

In [166]:
# Add trainlines on top of the same figure
p.multi_line('x', 'y', source=trsource, color="#B0B0B0", line_width=4, line_cap ='round', line_join = 'round')

GlyphRenderer(id='4817', ...)

In [167]:
# Add subway on top (as black points)
subway_v = p.circle('x', 'y', size=10, source=ssource, color="lime")
# Add stations on top (as blue hollow points)
stations_v = p.circle('x', 'y', size=10, source=stsource,  line_color="#3288bd", fill_color="white", line_width=3)

In [181]:
# Hover tool referring to our own data field using @ and
## a position on the graph using $
# adding hover only to subway and railway stations
hover = HoverTool( tooltips = [('Name ', '@name'),('Price','@mean'), ('Year','@year')]) #,('(x,y)', '($x, $y)')])

p.add_tools(hover)
p.hover.renderers=[subway_v, stations_v]


In [169]:
#removing grid lines

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

In [170]:
#hiding axis
p.axis.visible = False

In [171]:
# Set autohide to true to only show the toolbar when mouse is over plot
p.toolbar.autohide = True

In [172]:
# Output filepath to HTML
#output_file = r"./Bokeh_maps/simple_schematics_map.html"

# Save the map
#save(p, output_file);

In [173]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # set the `yr` name to `slider.value` and `source.data = new_data`
    year = slider.value
    new_data = {
        'Name'       : stsource.loc['name'],
        'Price':    stsource.loc['mean'],
        'Year':stsourcs.loc['year']
    }
    stsource.data = new_data

In [174]:
# Make a slider object: slider
slider = Slider(title="year", value=2012, start=2008, end=2019, step=1)

In [175]:
# Attach the callback to the 'value' property of slider
slider.on_change('value',update_plot)

In [182]:
show(p)

In [177]:
# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = row(widgetbox(slider), p)
curdoc().add_root(layout)

In [178]:
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



from bokeh.models.widgets import CheckboxGroup
# Create the checkbox selection element, available carriers is a  
# list of all airlines in the data
#labels =( np.unique(stsource.data['year'])).tolist()


string_years = [str(year) for year in stsource.data['year']] 
year_selection = CheckboxGroup(labels= string_years ,active = [0, 1])

# Link a change in selected buttons to the update function
#year_selection.on_change('active', update)

# Select the airlines names from the selection values
#

# Update function takes three default parameters
def update(attr, old, new):
    # Get the list of carriers for the graph
    years_to_plot = [year_selection.labels[i] for i in 
                        year_selection.active]
    # Make a new dataset based on the selected carriers and the 
    # make_dataset function defined earlier
    new_src = make_dataset(years_to_plot,
                           range_start = 2008,
                           range_end = 2018,
                           bin_width = 1)
    # Update the source used in the quad glpyhs
    stsource.data.update(new_src.data)

# Link a change in selected buttons to the update function
year_selection.on_change('active', update)

# Create a row layout
layout = row(year_selection, p)

In [115]:
from bokeh.models import Panel
from bokeh.models.widgets import Tabs
# Make a tab with the layout 
tabs = Tabs(tabs=[tab])

show(tabs)

In [127]:
# Make a slider object: slider
year = Slider(title="year", value=2012, start=2008, end=2019, step=1)

In [128]:
# Create Column Data Source that will be used by the plot
#in my case it's stsource